In [1]:
# import everything
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras import Sequential, optimizers
from keras.layers import Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard, EarlyStopping


np.random.seed(1)
tf.random.set_seed(1)

In [2]:
# load the data from numpy objects
X_train = np.load('../my_dataset/X_train.npy')
X_test = np.load('../my_dataset/X_test.npy')
X_validate = np.load('../my_dataset/X_validate.npy')

y_train = np.load('../my_dataset/y_train.npy')
y_test = np.load('../my_dataset/y_test.npy')
y_validate = np.load('../my_dataset/y_validate.npy')

In [ ]:
conv_layers = [3, 4]
conv_nodes = [32, 48]
kernel_dimensions = [3, 5]
epochs = 400
early_stops = 20

dropouts = [0, 0.05, 0.1, 0.15, 0.2]
batches = [8, 16, 24, 32]
repeats = 20

for conv_layer in conv_layers:
    for conv_node in conv_nodes:
        for dim in kernel_dimensions:
            for batch in batches:
                for dropout in dropouts:
                    for repeat in range(repeats):
                        NAME = "Conv{}Nodes{}Dim{}Batch{}Dropout{}No{}".format(conv_layer, conv_node, dim, batch, dropout, repeat)
                        print(NAME)

                        model = Sequential()

                        model.add(Conv2D(conv_node, (dim, dim), input_shape=X_train.shape[1:]))
                        model.add(Activation('relu'))
                        model.add(MaxPooling2D(pool_size=(2, 2)))
                        model.add(Dropout(dropout))

                        for l in range(conv_layer-1):
                            model.add(Conv2D(conv_node, (3, 3)))
                            model.add(Activation('relu'))
                            model.add(MaxPooling2D(pool_size=(2, 2)))
                            model.add(Dropout(dropout))

                        model.add(Flatten())

                        # output layer (as big as the number of words we teach)
                        output_neurons = len(np.unique(y_train))
                        model.add(tf.keras.layers.Dense(output_neurons, activation=tf.nn.softmax))

                        tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))


                        model.compile(loss='sparse_categorical_crossentropy',
                                    optimizer='adam',
                                    metrics=['accuracy'],
                                    )

                        callbacks = [EarlyStopping(monitor='val_accuracy', patience=early_stops, restore_best_weights=True), tensorboard]

                        model.fit(X_train, y_train,
                                epochs=epochs,
                                validation_data=(X_validate, y_validate),
                                callbacks=callbacks,
                                batch_size=batch)

                            # save the model
                        model.save("models/"+NAME, save_format="h5")

In [3]:
import os
conv_layers = [3, 4]
conv_nodes = [32, 48]
kernel_dimensions = [3, 5]
epochs = 400
early_stops = 20

dropouts = [0, 0.05, 0.1, 0.15, 0.2]
batches = [8, 16, 24, 32]

topFiveScores = [0, 0, 0, 0, 0]
topFiveNames = ['', '', '', '', '']
repeats = 20

for conv_layer in conv_layers:
    for conv_node in conv_nodes:
        for dim in kernel_dimensions:
            for batch in batches:
                for dropout in dropouts:
                    for repeat in range(repeats):
                        NAME = "Conv{}Nodes{}Dim{}Batch{}Dropout{}No{}".format(conv_layer, conv_node, dim, batch, dropout, repeat)

                        if not os.path.exists("models/"+NAME):
                            continue

                        model = tf.keras.models.load_model("models/"+NAME)
                        valuation = model.evaluate(X_validate, y_validate)[1]

                        for i in range(5):
                            if valuation > topFiveScores[i]:
                                topFiveScores[i] = valuation
                                topFiveNames[i] = NAME
                                break

for i in range(5):
    print(topFiveNames[i] + ": " + str(topFiveScores[i]))

14/14 [==============================] - 0s 10ms/step - loss: 3.4018 - accuracy: 0.0297
Conv4Nodes48Dim3Batch8Dropout0.2No2: 0.6453089118003845
Conv4Nodes32Dim3Batch32Dropout0.15No9: 0.6384439468383789
Conv4Nodes32Dim3Batch32Dropout0.2No18: 0.6338672637939453
Conv4Nodes48Dim3Batch16Dropout0.15No2: 0.6292906403541565
Conv4Nodes48Dim3Batch16Dropout0.2No10: 0.6247139573097229


In [4]:
for i in range(5):
    print(topFiveNames[i] + ": " + str(topFiveScores[i]))

Conv4Nodes48Dim3Batch8Dropout0.2No2: 0.6453089118003845
Conv4Nodes32Dim3Batch32Dropout0.15No9: 0.6384439468383789
Conv4Nodes32Dim3Batch32Dropout0.2No18: 0.6338672637939453
Conv4Nodes48Dim3Batch16Dropout0.15No2: 0.6292906403541565
Conv4Nodes48Dim3Batch16Dropout0.2No10: 0.6247139573097229
